In [495]:
import numpy as np #para processamento numérico em Python
import matplotlib.pyplot as plt #para os gráficos
import geopandas as gpd #para a leitura de shapefiles
import seaborn as sns #para fazer o mapa de kernel
from matplotlib.patches import Ellipse #para projetar a elipse 
import contextily
import pandas as pd

# pointpats --> análise espacial de pontos
from pointpats.centrography import hull, mbr, mean_center, weighted_mean_center, manhattan_median, std_distance,euclidean_median,ellipse
from pointpats import PointPattern
import pointpats.quadrat_statistics as qs
from pointpats.distance_statistics import G, F, K, Genv, Fenv, Kenv
from pointpats import PoissonPointProcess

# Municípios do Mato Grosso do Sul

In [496]:
ms = gpd.read_file('data/MS/ms.shp')

In [497]:
ms

,ID,CD_GEOCODM,NM_MUNICIP,geometry
0,140,5000203,ÁGUA CLARA,"POLYGON ((-53.09300 -20.21272, -53.08960 -20.2..."
1,141,5000252,ALCINÓPOLIS,"POLYGON ((-53.50659 -18.40831, -53.50794 -18.4..."
2,142,5000609,AMAMBAI,"POLYGON ((-54.92462 -22.92414, -54.92290 -22.9..."
3,143,5000708,ANASTÁCIO,"POLYGON ((-55.41798 -20.86286, -55.41794 -20.8..."
4,144,5000807,ANAURILÂNDIA,"POLYGON ((-52.73613 -21.85311, -52.70559 -21.8..."
...,...,...,...,...
73,213,5007950,TACURU,"POLYGON ((-55.20586 -23.40518, -55.20386 -23.4..."
74,214,5007976,TAQUARUSSU,"POLYGON ((-53.43801 -22.48280, -53.43799 -22.4..."
75,215,5008008,TERENOS,"POLYGON ((-54.76317 -20.47306, -54.76272 -20.4..."
76,216,5008305,TRÊS LAGOAS,"POLYGON ((-52.50132 -19.62639, -52.49514 -19.6..."


In [498]:
ms.crs

<Geographic 2D CRS: EPSG:4674>
Name: SIRGAS 2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Latin America - Central America and South America - onshore and offshore. Brazil - onshore and offshore.
- bounds: (-122.19, -59.87, -25.28, 32.72)
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [499]:
def calcula_centroide(geometry):
  return geometry.centroid

In [500]:
ms["centroides"] = ms["geometry"].apply(lambda polygon: calcula_centroide(polygon))

# Concatenando Tabelas

In [501]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
ls_df = [
pd.read_csv('data/ibge/Tabela616 - População economicamente ativa - MUNICIPIOS.csv', sep=',', skiprows=5),             # Done
pd.read_csv('data/ibge/Tabela1384 - Pessoas por classes de rendimento - MUNICIPIOS.csv', sep=',', skiprows=4),        # Done
pd.read_csv('data/ibge/Tabela1394 - Domicílios por condição de ocupação - MUNICIPIOS.csv', sep=',', skiprows=6),      # Done
pd.read_csv('data/ibge/Tabela1394 - Domicílios por número de banheiros - MUNICIPIOS.csv', sep=',', skiprows=5),     # Done
pd.read_csv('data/ibge/Tabela1552 - Residentes com 65 anos ou mais - MUNICIPIOS.csv', sep=';', skiprows=7),        # Unnamed columns
pd.read_csv('data/ibge/Tabela1552 - Residentes menores de 18 anos - MUNICIPIOS.csv', sep=';', skiprows=7),         # Unnamed columns
pd.read_csv('data/ibge/Tabela2009 - Domicílios por número de dormitórios - MUNICIPIOS.csv', sep=',', skiprows=4),     # Done
pd.read_csv('data/ibge/Tabela2031 - Pessoas por posição na ocupação e categoria do emprego no trabalho principal - MUNICIPIOS.csv', sep=',', skiprows=4), #Done
pd.read_csv('data/ibge/Tabela3219 - Domicílios por número de moradores - MUNICIPIOS.csv', sep=',', skiprows=4),       # Done
pd.read_csv('data/ibge/Tabela3219 - Domicílios por tipo - MUNICIPIOS.csv', sep=',', skiprows=6),                 # Done
pd.read_csv('data/ibge/Tabela3277 - Pessoas por classes de rendimentos - MUNICIPIOS CENTRO OESTE.csv', sep=',', skiprows=4), #Done
pd.read_csv('data/ibge/Tabela3573 - Pessoas desocupadas.csv', sep=',', skiprows=6),
pd.read_csv('data/ibge/Tabela3603 - Pessoas ocupadas por local do trabalho principal.csv', sep=',', skiprows=4),
pd.read_csv('data/ibge/Tabela3604 - Pessoas ocupadas por tempo de deslocamento para trabalho.csv', sep=',', skiprows=4),
]

ls_names = ['arquivo_616',
'arquivo_1384',
'arquivo_1394-o',
'arquivo_1394-b',
'arquivo_1552-65',
'arquivo_1552-18',
'arquivo_2009',
'arquivo_2031',
'arquivo_3219',
'arquivo_3219-tipo',
'arquivo_3277',
'arquivo_3573',
'arquivo_3603',
'arquivo_3604']





In [502]:
ls_df[9]

,Cód.,Município,Condição de ocupação do domicílio,Existência de banheiro ou sanitário e esgotamento sanitário,Total,Casa,Casa de vila ou em condomínio,Apartamento,"Habitação em casa de cômodos, cortiço ou cabeça de porco",Oca ou maloca
0,1100015,Alta Floresta D'Oeste (RO),Total,Total,7270.0,7244.0,9,15,1,1
1,1100023,Ariquemes (RO),Total,Total,27166.0,24734.0,168,2236,28,-
2,1100031,Cabixi (RO),Total,Total,1975.0,1973.0,1,1,-,-
3,1100049,Cacoal (RO),Total,Total,24215.0,23012.0,14,1161,27,1
4,1100056,Cerejeiras (RO),Total,Total,5348.0,5303.0,6,34,5,-
...,...,...,...,...,...,...,...,...,...,...
5573,0,Zero resultante de um cálculo ou arredondament...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5574,X,Valor inibido para não identificar o informant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,..,Valor não se aplica.\nEx: Não se pode obter o ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5576,...,Valor não disponível.\nEx: A produção de feijã...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Remover cidades nao interessantes:

In [503]:
for i, df in enumerate(ls_df):
    ls_df[i] = pd.merge(df, ms['CD_GEOCODM'], left_on='Cód.', right_on='CD_GEOCODM', how='inner').drop(columns=['CD_GEOCODM'])

## Tratamentos especiais:


In [504]:
ls_df[6] = ls_df[6].pivot(index=['Cód.', 'Município'], columns='Número de dormitórios', values='Total').reset_index()
ls_df[2] = ls_df[2].pivot(index=['Cód.', 'Município'], columns='Condição de ocupação do domicílio', values='Total').reset_index()
ls_df[4].columns = ["Cód.", "Município", "Total de pessoas com 65 anos ou mais"]
ls_df[5].columns = ["Cód.", "Município", "Total de pessoas com 18 anos ou menos"]


In [505]:
# Lidando com tabela de idade, renda
cols = ls_df[10].columns[3:]
temp = ls_df[10][cols].set_index('Grupo de idade', append=True).unstack().sort_index(axis=1)
temp.columns = temp.columns.map(lambda x: f'{x[0]}_{x[1]}')
temp2 = ls_df[10].drop(cols, axis=1).join(temp)
temp2.drop(columns=['Sexo'], inplace=True)
ls_df[10] = temp2.groupby(['Cód.', 'Município']).sum().reset_index()

In [506]:
# ls_df[2]
# # Get percentage off all columns from Total
# for i, df in enumerate(ls_df):

## Renomeando colunas

In [507]:
for i, df in enumerate(ls_df):
    df.name = ls_names[i]

In [508]:
def rename_columns(df):
    num_table = df.name.split('_')[1]
    columns = [f'[{num_table}]{col}' for col in df.columns[2:]]
    columns.insert(0, 'Municípios')
    columns.insert(0, 'Cod Mun')
    df.columns = columns
    return df

In [509]:
for df in ls_df:
    df = rename_columns(df)


In [510]:
# Print shape of list of dataframes
sum = 0
for i in range(len(ls_df)):
    print(ls_df[i].shape)
    sum += ls_df[i].shape[1]
print(f"Numero total de colunas esperado: {sum-(len(ls_df)-1)*2}")

(78, 5)
(78, 12)
(78, 7)
(78, 11)
(78, 3)
(78, 3)
(78, 9)
(78, 10)
(78, 19)
(78, 10)
(78, 198)
(78, 5)
(78, 10)
(78, 8)
Numero total de colunas esperado: 284


In [511]:
# Checa quantos valores nulos tem em cada dataframe
for i, df in enumerate(ls_df):
    # print()
    # print(ls_names[i])
    # print(df.isna().sum())
    # Drop columns that rows contain the string "Total"
    ls_df[i] = df.loc[:, ~(df == 'Total').any()]
    ls_df[i] = ls_df[i].replace('-', 0)

## Normaliza dados de domicilio:


In [512]:
# Convert all columns to numeric
for i, df in enumerate(ls_df):
    ls_df[i] = df.apply(pd.to_numeric, errors='ignore')

In [513]:
def get_percentage(df):
    cols = df.columns[2:]
    total_col = [col for col in df.columns if 'Total' in col]
    # All columns except total_col
    cols = [col for col in cols if col not in total_col]
    for col in cols:
        df[col] = df[col] / df[total_col[0]]
    return df

tables_to_apply = [1, 2, 3, 6, 7, 8, 9, 12]

for i in tables_to_apply:
    ls_df[i] = get_percentage(ls_df[i])


## Lidando com tabela 3277:

In [515]:
def get_percentage_3277(df):
    cols = df.columns[2:]
    ls_cols = [
    [col for col in df.columns if '10 a 14 anos' in col],
    [col for col in df.columns if '15 a 17 anos' in col],
    [col for col in df.columns if '15 a 19 anos' in col],
    [col for col in df.columns if '18 ou 19 anos' in col],
    [col for col in df.columns if '20 a 24 anos' in col],
    [col for col in df.columns if '25 a 29 anos' in col],
    [col for col in df.columns if '30 a 34 anos' in col],
    [col for col in df.columns if '35 a 39 anos' in col],
    [col for col in df.columns if '40 a 44 anos' in col],
    [col for col in df.columns if '45 a 49 anos' in col],
    [col for col in df.columns if '50 a 54 anos' in col],
    [col for col in df.columns if '55 a 59 anos' in col],
    [col for col in df.columns if '60 a 69 anos' in col],
    [col for col in df.columns if '70 anos ou mais' in col]
    ]
    for age_group in ls_cols:
        
        total_col = [col for col in age_group if 'Total' in col]
        # All columns except total_col
        cols = [col for col in age_group if col not in total_col]
        for col in cols:
            df[col] = df[col] / df[total_col[0]]

    return df

get_percentage_3277(ls_df[10])

,Cod Mun,Municípios,[3277]Até 1/4 de salário mínimo_10 a 14 anos,[3277]Até 1/4 de salário mínimo_15 a 17 anos,[3277]Até 1/4 de salário mínimo_15 a 19 anos,[3277]Até 1/4 de salário mínimo_18 ou 19 anos,[3277]Até 1/4 de salário mínimo_20 a 24 anos,[3277]Até 1/4 de salário mínimo_25 a 29 anos,[3277]Até 1/4 de salário mínimo_30 a 34 anos,[3277]Até 1/4 de salário mínimo_35 a 39 anos,[3277]Até 1/4 de salário mínimo_40 a 44 anos,[3277]Até 1/4 de salário mínimo_45 a 49 anos,[3277]Até 1/4 de salário mínimo_50 a 54 anos,[3277]Até 1/4 de salário mínimo_55 a 59 anos,[3277]Até 1/4 de salário mínimo_60 a 69 anos,[3277]Até 1/4 de salário mínimo_70 anos ou mais,[3277]Mais de 1 a 2 salários mínimos_10 a 14 anos,[3277]Mais de 1 a 2 salários mínimos_15 a 17 anos,[3277]Mais de 1 a 2 salários mínimos_15 a 19 anos,[3277]Mais de 1 a 2 salários mínimos_18 ou 19 anos,[3277]Mais de 1 a 2 salários mínimos_20 a 24 anos,[3277]Mais de 1 a 2 salários mínimos_25 a 29 anos,[3277]Mais de 1 a 2 salários mínimos_30 a 34 anos,[3277]Mais de 1 a 2 salários mínimos_35 a 39 anos,[3277]Mais de 1 a 2 salários mínimos_40 a 44 anos,[3277]Mais de 1 a 2 salários mínimos_45 a 49 anos,[3277]Mais de 1 a 2 salários mínimos_50 a 54 anos,[3277]Mais de 1 a 2 salários mínimos_55 a 59 anos,[3277]Mais de 1 a 2 salários mínimos_60 a 69 anos,[3277]Mais de 1 a 2 salários mínimos_70 anos ou mais,[3277]Mais de 1/2 a 1 salário mínimo_10 a 14 anos,[3277]Mais de 1/2 a 1 salário mínimo_15 a 17 anos,[3277]Mais de 1/2 a 1 salário mínimo_15 a 19 anos,[3277]Mais de 1/2 a 1 salário mínimo_18 ou 19 anos,[3277]Mais de 1/2 a 1 salário mínimo_20 a 24 anos,[3277]Mais de 1/2 a 1 salário mínimo_25 a 29 anos,[3277]Mais de 1/2 a 1 salário mínimo_30 a 34 anos,[3277]Mais de 1/2 a 1 salário mínimo_35 a 39 anos,[3277]Mais de 1/2 a 1 salário mínimo_40 a 44 anos,[3277]Mais de 1/2 a 1 salário mínimo_45 a 49 anos,[3277]Mais de 1/2 a 1 salário mínimo_50 a 54 anos,[3277]Mais de 1/2 a 1 salário mínimo_55 a 59 anos,[3277]Mais de 1/2 a 1 salário mínimo_60 a 69 anos,[3277]Mais de 1/2 a 1 salário mínimo_70 anos ou mais,[3277]Mais de 1/4 a 1/2 salário mínimo_10 a 14 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_15 a 17 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_15 a 19 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_18 ou 19 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_20 a 24 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_25 a 29 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_30 a 34 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_35 a 39 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_40 a 44 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_45 a 49 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_50 a 54 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_55 a 59 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_60 a 69 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_70 anos ou mais,[3277]Mais de 10 a 15 salários mínimos_10 a 14 anos,[3277]Mais de 10 a 15 salários mínimos_15 a 17 anos,[3277]Mais de 10 a 15 salários mínimos_15 a 19 anos,[3277]Mais de 10 a 15 salários mínimos_18 ou 19 anos,[3277]Mais de 10 a 15 salários mínimos_20 a 24 anos,[3277]Mais de 10 a 15 salários mínimos_25 a 29 anos,[3277]Mais de 10 a 15 salários mínimos_30 a 34 anos,[3277]Mais de 10 a 15 salários mínimos_35 a 39 anos,[3277]Mais de 10 a 15 salários mínimos_40 a 44 anos,[3277]Mais de 10 a 15 salários mínimos_45 a 49 anos,[3277]Mais de 10 a 15 salários mínimos_50 a 54 anos,[3277]Mais de 10 a 15 salários mínimos_55 a 59 anos,[3277]Mais de 10 a 15 salários mínimos_60 a 69 anos,[3277]Mais de 10 a 15 salários mínimos_70 anos ou mais,[3277]Mais de 15 a 20 salários mínimos_10 a 14 anos,[3277]Mais de 15 a 20 salários mínimos_15 a 17 anos,[3277]Mais de 15 a 20 salários mínimos_15 a 19 anos,[3277]Mais de 15 a 20 salários mínimos_18 ou 19 anos,[3277]Mais de 15 a 20 salários mínimos_20 a 24 anos,[3277]Mais de 15 a 20 salários mínimos_25 a 29 anos,[3277]Mais de 15 a 20 salários mínimos_30 a 34 anos,[3277]Mais de 15 a 20 salários mínimos_35 a 39 anos,[3277]Mais de 15 a 20 salários mínimos_40 a 44 anos,[3277

## Tudo em um dataframe

In [516]:
# Merge list of dataframes
total_df = ls_df[0]
for i in range(1, len(ls_df)):
    total_df = total_df.merge(ls_df[i].loc[:, ls_df[i].columns != 'Municípios'], on="Cod Mun", how="left")
total_df.shape

(78, 272)

In [517]:
total_df.dtypes

Cod Mun                                    int64
Municípios                                object
[616]Total                               float64
[1384]Total                              float64
[1384]Até 1/2 salário mínimo             float64
                                          ...   
[3604]Até cinco minutos                    int64
[3604]De seis minutos até meia hora        int64
[3604]Mais de meia hora até uma hora       int64
[3604]Mais de uma hora até duas horas      int64
[3604]Mais de duas horas                   int64
Length: 272, dtype: object

In [518]:
ms.dtypes

ID               int64
CD_GEOCODM      object
NM_MUNICIP      object
geometry      geometry
centroides    geometry
dtype: object

In [519]:
ms = ms.apply(pd.to_numeric, errors='ignore')

In [520]:
final_df = total_df.merge(ms, left_on='Cod Mun', right_on='CD_GEOCODM', how='inner').drop(columns=['CD_GEOCODM', 'ID', 'NM_MUNICIP'])

In [521]:
final_df

,Cod Mun,Municípios,[616]Total,[1384]Total,[1384]Até 1/2 salário mínimo,[1384]Mais de 1/2 a 1 salário mínimo,[1384]Mais de 1 a 2 salários mínimos,[1384]Mais de 2 a 5 salários mínimos,[1384]Mais de 5 a 10 salários mínimos,[1384]Mais de 10 a 20 salários mínimos,[1384]Mais de 20 salários mínimos,[1384]Sem rendimento,[1384]Sem declaração,[1394-o]Alugado,[1394-o]Cedido,[1394-o]Outra condição,[1394-o]Próprio,[1394-o]Total,[1394-b]Total,[1394-b]Tinham banheiro de uso exclusivo do domicílio - 1 banheiro,[1394-b]Tinham banheiro de uso exclusivo do domicílio - 2 banheiros,[1394-b]Tinham banheiro de uso exclusivo do domicílio - 3 banheiros,[1394-b]Tinham banheiro de uso exclusivo do domicílio - 4 banheiros ou mais,[1394-b]Tinham sanitário,[1394-b]Não tinham banheiro nem sanitário,[1552-65]Total de pessoas com 65 anos ou mais,[1552-18]Total de pessoas com 18 anos ou menos,[2009]1 dormitório,[2009]2 dormitórios,[2009]3 dormitórios,[2009]4 dormitórios,[2009]5 dormitórios,[2009]6 dormitórios ou mais,[2009]Total,[2031]Total,[2031]Empregado - com carteira de trabalho assinada,[2031]Empregado - militar e funcionário público estatutário,[2031]Empregado - outro sem carteira de trabalho assinada,[2031]Não remunerado em ajuda a membro do domicílio,[2031]Trabalhador na produção para o próprio consumo,[2031]Empregador,[2031]Conta própria,[3219]Total,[3219]1 morador,[3219]2 moradores,[3219]3 moradores,[3219]4 moradores,[3219]5 moradores,[3219]6 moradores,[3219]7 moradores,[3219]8 moradores,[3219]9 moradores,[3219]10 moradores,[3219]11 moradores,[3219]12 moradores,[3219]13 moradores,[3219]14 moradores ou mais,[3219-tipo]Total,[3219-tipo]Casa,[3219-tipo]Casa de vila ou em condomínio,[3219-tipo]Apartamento,"[3219-tipo]Habitação em casa de cômodos, cortiço ou cabeça de porco",[3219-tipo]Oca ou maloca,[3277]Até 1/4 de salário mínimo_10 a 14 anos,[3277]Até 1/4 de salário mínimo_15 a 17 anos,[3277]Até 1/4 de salário mínimo_15 a 19 anos,[3277]Até 1/4 de salário mínimo_18 ou 19 anos,[3277]Até 1/4 de salário mínimo_20 a 24 anos,[3277]Até 1/4 de salário mínimo_25 a 29 anos,[3277]Até 1/4 de salário mínimo_30 a 34 anos,[3277]Até 1/4 de salário mínimo_35 a 39 anos,[3277]Até 1/4 de salário mínimo_40 a 44 anos,[3277]Até 1/4 de salário mínimo_45 a 49 anos,[3277]Até 1/4 de salário mínimo_50 a 54 anos,[3277]Até 1/4 de salário mínimo_55 a 59 anos,[3277]Até 1/4 de salário mínimo_60 a 69 anos,[3277]Até 1/4 de salário mínimo_70 anos ou mais,[3277]Mais de 1 a 2 salários mínimos_10 a 14 anos,[3277]Mais de 1 a 2 salários mínimos_15 a 17 anos,[3277]Mais de 1 a 2 salários mínimos_15 a 19 anos,[3277]Mais de 1 a 2 salários mínimos_18 ou 19 anos,[3277]Mais de 1 a 2 salários mínimos_20 a 24 anos,[3277]Mais de 1 a 2 salários mínimos_25 a 29 anos,[3277]Mais de 1 a 2 salários mínimos_30 a 34 anos,[3277]Mais de 1 a 2 salários mínimos_35 a 39 anos,[3277]Mais de 1 a 2 salários mínimos_40 a 44 anos,[3277]Mais de 1 a 2 salários mínimos_45 a 49 anos,[3277]Mais de 1 a 2 salários mínimos_50 a 54 anos,[3277]Mais de 1 a 2 salários mínimos_55 a 59 anos,[3277]Mais de 1 a 2 salários mínimos_60 a 69 anos,[3277]Mais de 1 a 2 salários mínimos_70 anos ou mais,[3277]Mais de 1/2 a 1 salário mínimo_10 a 14 anos,[3277]Mais de 1/2 a 1 salário mínimo_15 a 17 anos,[3277]Mais de 1/2 a 1 salário mínimo_15 a 19 anos,[3277]Mais de 1/2 a 1 salário mínimo_18 ou 19 anos,[3277]Mais de 1/2 a 1 salário mínimo_20 a 24 anos,[3277]Mais de 1/2 a 1 salário mínimo_25 a 29 anos,[3277]Mais de 1/2 a 1 salário mínimo_30 a 34 anos,[3277]Mais de 1/2 a 1 salário mínimo_35 a 39 anos,[3277]Mais de 1/2 a 1 salário mínimo_40 a 44 anos,[3277]Mais de 1/2 a 1 salário mínimo_45 a 49 anos,[3277]Mais de 1/2 a 1 salário mínimo_50 a 54 anos,[3277]Mais de 1/2 a 1 salário mínimo_55 a 59 anos,[3277]Mais de 1/2 a 1 salário mínimo_60 a 69 anos,[3277]Mais de 1/2 a 1 salário mínimo_70 anos ou mais,[3277]Mais de 1/4 a 1/2 salário mínimo_10 a 14 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_15 a 17 anos,[3277]Mais de 1/4 a 1/2 salário mínimo_15 a 19 anos

In [522]:
final_df.to_csv('MS.csv', index=False, sep=',')